# MWRR Activity
Integrantes: Marianne Trujillo ALtamirano, Ana Luisa Espinoza López.

In [96]:
import pyxirr as px
import pandas as pd
import dateparser as dp

In [97]:
data = "data_actividad.xlsx"
balances = pd.read_excel(data, sheet_name='balances')
movements = pd.read_excel(data, sheet_name='movements')

In [98]:
# 1. Función limpieza balances
def clean_balance_data(data):
    """
    Devuelve un diccionario {contrato: df_balance}
    Cada df tiene columnas: balance_date, portfolio_value
    """
    cols = ["contract", "value_pos_mdo", "balance_date"]
    df = data[cols].copy()

    # Parse fechas
    df["balance_date"] = df["balance_date"].apply(lambda x: dp.parse(str(x)))

    # Diccionario de balances por contrato
    contract_dfs = {}
    for contract, group in df.groupby("contract"):
        grouped = (
            group.groupby("balance_date", as_index=False)["value_pos_mdo"]
            .sum()
            .rename(columns={"value_pos_mdo": "portfolio_value"})
            .sort_values("balance_date")
        )
        contract_dfs[contract] = grouped.reset_index(drop=True)

    return contract_dfs

In [67]:
bal_clean = clean_balance_data(balances)
bal_clean.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Balance contrato 12861603

In [99]:
bal_clean['12861603'].head()

,balance_date,portfolio_value
0,2023-01-12,100000.0000
1,2023-01-13,100000.0000
2,2023-01-16,267081.1981
3,2023-01-17,270193.5784
4,2023-01-18,270369.3147


#### Balance contrato 20486403

In [100]:
bal_clean['20486403'].head()

,balance_date,portfolio_value
0,2022-12-09,1.531486e+06
1,2022-12-13,1.532354e+06
2,2022-12-14,1.533558e+06
3,2022-12-15,1.533234e+06
4,2022-12-16,1.533722e+06


#### Balance contrato AHA84901

In [101]:
bal_clean['AHA84901'].head()

,balance_date,portfolio_value
0,2023-06-26,200000.0000
1,2023-06-27,204868.6095
2,2023-06-28,204914.2665
3,2023-06-29,204976.7575
4,2023-06-30,205024.9605


In [102]:
# 2. Función limpieza movimientos
def clean_movements_data(df_mov):
    """
    Devuelve un diccionario {contrato: df_mov}
    Cada df tiene columnas: description, movement_import, operation_date
    """
    cols = ["contract", "movement_import", "operation_date", "description"]
    df = df_mov[cols].copy()

    # Parse fechas
    df["operation_date"] = df["operation_date"].apply(
        lambda x: dp.parse(str(x)) if pd.notnull(x) else pd.NaT
    )
    
    # Filtrar solo depósitos y retiros de efectivo (y transferencias)
    df_filtered = df[
        df["description"].str.contains("DEPOSITO DE EFECTIVO", case=False, na=False) |
        df["description"].str.contains("RETIRO DE EFECTIVO", case=False, na=False)
    ].copy()
    
    # Clasificar descripciones
    df_filtered["description_clean"] = df_filtered["description"].apply(
        lambda x: "DEPOSITO DE EFECTIVO" if "DEPOSITO" in x.upper() else "RETIRO DE EFECTIVO"
    )

    contract_dfs = {}
    for contract, group in df_filtered.groupby("contract"):
        # Seleccionar solo las columnas finales requeridas y ordenar por fecha
        movements_df = (group[["description_clean", "movement_import", "operation_date"]]
                       .copy()
                       .sort_values("operation_date"))
        contract_dfs[contract] = movements_df.reset_index(drop=True)

    return contract_dfs

In [103]:
mov_clean = clean_movements_data(movements)
mov_clean.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Movimientos 12861603

In [108]:
mov_clean['12861603'].head()

,description_clean,movement_import,operation_date
0,DEPOSITO DE EFECTIVO,100000.00,2023-01-12
1,DEPOSITO DE EFECTIVO,167000.00,2023-01-16
2,DEPOSITO DE EFECTIVO,3241.77,2023-01-17
3,DEPOSITO DE EFECTIVO,10000.00,2023-02-01
4,DEPOSITO DE EFECTIVO,10000.00,2023-02-10


#### Movimientos contrato 20486403

In [110]:
mov_clean['20486403'].head()

,description_clean,movement_import,operation_date
0,RETIRO DE EFECTIVO,26000.0,2023-01-04
1,RETIRO DE EFECTIVO,1500.0,2023-01-25
2,DEPOSITO DE EFECTIVO,30000.0,2023-03-03
3,RETIRO DE EFECTIVO,23500.0,2023-04-28
4,DEPOSITO DE EFECTIVO,80000.0,2023-05-02


#### Movimientos contrato AHA84901

In [112]:
mov_clean['AHA84901'].head()

,description_clean,movement_import,operation_date
0,DEPOSITO DE EFECTIVO,200000.00,2023-06-26
1,DEPOSITO DE EFECTIVO,5000.00,2023-06-27
2,DEPOSITO DE EFECTIVO,340000.00,2023-07-24
3,RETIRO DE EFECTIVO,126.30,2023-08-15
4,RETIRO DE EFECTIVO,229.03,2023-09-14


In [113]:
# 3. Función para agregar valores inicial y final
def add_initial_final_values(bal_clean, mov_clean):
    """
    Agrega valor inicial y final a los movimientos de cada contrato
    Devuelve un diccionario {contrato: df_mov_completo}
    """
    complete_data = {}
    
    for contract in mov_clean.keys():
        if contract not in bal_clean:
            continue
            
        # Obtener datos de movimientos y balances del contrato
        mov_df = mov_clean[contract].copy()
        bal_df = bal_clean[contract].copy()
        
        if len(bal_df) == 0 or len(mov_df) == 0:
            continue
        
        # Obtener primer balance y primer movimiento
        first_balance = bal_df.iloc[0]['portfolio_value']
        first_movement = mov_df.iloc[0]['movement_import']
        first_balance_date = bal_df.iloc[0]['balance_date']
        
        # Decidir valor inicial
        if abs(first_movement - first_balance) < 0.01:  # Son iguales (tolerancia pequeña)
            # Renombrar el primer movimiento como VALOR_INICIAL
            mov_df.iloc[0, mov_df.columns.get_loc('description_clean')] = 'VALOR_INICIAL'
            mov_df_final = mov_df.copy()
        else:
            # Agregar valor inicial del balance al inicio
            initial_row = pd.DataFrame({
                'description_clean': ['VALOR_INICIAL'],
                'movement_import': [first_balance],
                'operation_date': [first_balance_date]
            })
            mov_df_final = pd.concat([initial_row, mov_df], ignore_index=True)
        
        # Valor final (último balance)
        final_value = bal_df.iloc[-1]
        final_row = pd.DataFrame({
            'description_clean': ['VALOR_FINAL'],
            'movement_import': [final_value['portfolio_value']],
            'operation_date': [final_value['balance_date']]
        })
        
        # Ajustar signos de movimientos: depósitos negativos, retiros positivos
        mov_df_adjusted = mov_df_final.copy()
        for idx, row in mov_df_adjusted.iterrows():
            if row['description_clean'] == 'DEPOSITO DE EFECTIVO':
                mov_df_adjusted.at[idx, 'movement_import'] = -abs(row['movement_import'])
            elif row['description_clean'] == 'RETIRO DE EFECTIVO':
                mov_df_adjusted.at[idx, 'movement_import'] = abs(row['movement_import'])
            elif row['description_clean'] == 'VALOR_INICIAL':
                mov_df_adjusted.at[idx, 'movement_import'] = -abs(row['movement_import'])
        
        # Combinar todos los datos
        complete_df = pd.concat([mov_df_adjusted, final_row], ignore_index=True)
        complete_df = complete_df.sort_values('operation_date').reset_index(drop=True)
        
        complete_data[contract] = complete_df
    
    return complete_data

In [114]:
complete_data = add_initial_final_values(bal_clean, mov_clean)
complete_data.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Flujos contrato 12861603

In [115]:
complete_data['12861603'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-100000.00,2023-01-12
1,DEPOSITO DE EFECTIVO,-167000.00,2023-01-16
2,DEPOSITO DE EFECTIVO,-3241.77,2023-01-17
3,DEPOSITO DE EFECTIVO,-10000.00,2023-02-01
4,DEPOSITO DE EFECTIVO,-10000.00,2023-02-10


In [117]:
complete_data['12861603'].tail()

,description_clean,movement_import,operation_date
12,RETIRO DE EFECTIVO,100.1700,2023-10-13
13,RETIRO DE EFECTIVO,106.1600,2023-11-15
14,RETIRO DE EFECTIVO,5000.0000,2023-11-17
15,RETIRO DE EFECTIVO,30000.0000,2023-11-30
16,VALOR_FINAL,223059.8805,2023-11-30


#### Flujos contrato 20486403

In [118]:
complete_data['20486403'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-1.531486e+06,2022-12-09
1,RETIRO DE EFECTIVO,2.600000e+04,2023-01-04
2,RETIRO DE EFECTIVO,1.500000e+03,2023-01-25
3,DEPOSITO DE EFECTIVO,-3.000000e+04,2023-03-03
4,RETIRO DE EFECTIVO,2.350000e+04,2023-04-28


In [120]:
complete_data['12861603'].tail()

,description_clean,movement_import,operation_date
12,RETIRO DE EFECTIVO,100.1700,2023-10-13
13,RETIRO DE EFECTIVO,106.1600,2023-11-15
14,RETIRO DE EFECTIVO,5000.0000,2023-11-17
15,RETIRO DE EFECTIVO,30000.0000,2023-11-30
16,VALOR_FINAL,223059.8805,2023-11-30


#### Flujos contrato AHA84901

In [121]:
complete_data['AHA84901'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-200000.00,2023-06-26
1,DEPOSITO DE EFECTIVO,-5000.00,2023-06-27
2,DEPOSITO DE EFECTIVO,-340000.00,2023-07-24
3,RETIRO DE EFECTIVO,126.30,2023-08-15
4,RETIRO DE EFECTIVO,229.03,2023-09-14


In [123]:
complete_data['AHA84901'].tail()

,description_clean,movement_import,operation_date
6,RETIRO DE EFECTIVO,30000.0000,2023-10-19
7,RETIRO DE EFECTIVO,5000.0000,2023-10-26
8,RETIRO DE EFECTIVO,227.5000,2023-11-15
9,RETIRO DE EFECTIVO,100000.0000,2023-11-23
10,VALOR_FINAL,430842.6591,2023-11-30


In [124]:
# 4. Función para calcular MWRR
def calculate_mwrr_by_contract(complete_data):
    """
    Calcula el MWRR (TIR) para cada contrato usando pyxirr
    Devuelve un diccionario {contrato: mwrr}
    """
    mwrr_results = {}
    
    for contract, df in complete_data.items():
        try:
            dates = df['operation_date'].tolist()
            amounts = df['movement_import'].tolist()
            
            # Calcular MWRR usando pyxirr
            mwrr = px.xirr(dates, amounts)
            mwrr_results[contract] = mwrr
            
        except Exception as e:
            print(f"Error calculando MWRR para contrato {contract}: {e}")
            mwrr_results[contract] = None
    
    return mwrr_results

#### Resultados

In [125]:
mwrr_by_contract = calculate_mwrr_by_contract(complete_data)

print("MWRR por contrato:")
for contract, mwrr in mwrr_by_contract.items():
    if mwrr is not None:
        print(f"{contract}: {mwrr:.2%}")
    else:
        print(f"{contract}: Error en cálculo")

MWRR por contrato:
12861603: 11.69%
20486403: 11.53%
AHA84901: 11.05%
